## Import Required Packages

In [12]:
import sys
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt
%matplotlib inline

import time
from datetime import datetime

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate, Dropout
from keras.models import Model

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K

%load_ext tensorboard

from sklearn.utils import shuffle

import numpy.random as rng
np.random.seed(1337)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Create Data Generators

In [18]:
SIZE = 105
IMG_SIZE = (SIZE, SIZE)
NUM_CHANNELS = 3
BATCH_SIZE = 16

# Define training augmentations
train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    shear_range=0.2)


# Define testing augmentations
test_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
# Create example augmented images
img = load_img('F:/Work/University/Year 5/ACS6420_Advanced Project/Code/Data/omniglot/images_background/images_background/Armenian/character01/0027_01.png')
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

i = 0
for batch in train_datagen.flow(x, batch_size=1, save_to_dir='F:/Work/University/Year 5/ACS6420_Advanced Project/Code/Data/preview', save_format='jpeg'):
    i+=1
    if i > 9:
        break

In [32]:
train_generator = train_datagen.flow_from_directory(
    'F:/Work/University/Year 5/ACS6420_Advanced Project/Code/Data/omniglot/train_small/',
    target_size=IMG_SIZE,
    color_mode='rgb',
    shuffle=True,
    seed=1,
    batch_size=BATCH_SIZE,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    'F:/Work/University/Year 5/ACS6420_Advanced Project/Code/Data/omniglot/val_small/',
    target_size=IMG_SIZE,
    color_mode='rgb',
    shuffle=True,
    seed=2,
    batch_size=BATCH_SIZE,
    class_mode='binary')

Found 200 images belonging to 2 classes.
Found 96 images belonging to 2 classes.


## Create Convolutional Neural Network

In [33]:
base_logdir = "logs/scalars/"
for f in os.listdir(base_logdir):
    file_path = os.path.join(base_logdir, f)
    
    shutil.rmtree(file_path)

logdir = "logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [34]:
# Convolutional Neural Network
model = VGG16(include_top=False, input_shape=(SIZE, SIZE, NUM_CHANNELS))

# Only retrain last block of VGG16 - 3 conv layers
for layer in model.layers:
    #if not layer.name.startswith('block5'):
    layer.trainable = False

# Add own layers at end for task
flat1 = Flatten()(model.layers[-1].output)
dense1 = Dense(32, activation='relu', kernel_regularizer=l2(1E-4), bias_regularizer=l2(1E-3))(flat1)
dropout1 = Dropout(0.5)(dense1)

output = Dense(1, activation='sigmoid')(dropout1)

model = Model(inputs=model.inputs, outputs=output)

model.summary()

optimizer = Adam(lr=0.0001)
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy', 'Precision', 'Recall'])

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 105, 105, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 105, 105, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 105, 105, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 52, 52, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 52, 52, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 52, 52, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 26, 26, 128)      

In [35]:
model.fit(train_generator, steps_per_epoch=train_generator.samples//BATCH_SIZE, 
          validation_data=validation_generator, validation_steps=validation_generator.samples//BATCH_SIZE,
          epochs=75, callbacks=[tensorboard_callback], verbose=1)

Epoch 1/75
12/12 [==============================] - 2s 159ms/step - loss: 0.7561 - accuracy: 0.5217 - precision: 0.5181 - recall: 0.4725 - val_loss: 0.6269 - val_accuracy: 0.6771 - val_precision: 0.6889 - val_recall: 0.6458
Epoch 2/75
12/12 [==============================] - 1s 48ms/step - loss: 0.6729 - accuracy: 0.5815 - precision: 0.5701 - recall: 0.6630 - val_loss: 0.5978 - val_accuracy: 0.6979 - val_precision: 0.7209 - val_recall: 0.6458
Epoch 3/75
12/12 [==============================] - 1s 50ms/step - loss: 0.6644 - accuracy: 0.5938 - precision: 0.6000 - recall: 0.5625 - val_loss: 0.5939 - val_accuracy: 0.6979 - val_precision: 0.8276 - val_recall: 0.5000
Epoch 4/75
12/12 [==============================] - 1s 48ms/step - loss: 0.6304 - accuracy: 0.6576 - precision: 0.6747 - recall: 0.6087 - val_loss: 0.5782 - val_accuracy: 0.7188 - val_precision: 0.7838 - val_recall: 0.6042
Epoch 5/75
12/12 [==============================] - 1s 48ms/step - loss: 0.5695 - accuracy: 0.7500 - precis

12/12 [==============================] - 1s 48ms/step - loss: 0.4182 - accuracy: 0.8587 - precision: 0.8824 - recall: 0.8242 - val_loss: 0.4417 - val_accuracy: 0.8125 - val_precision: 0.8409 - val_recall: 0.7708
Epoch 37/75
12/12 [==============================] - 1s 50ms/step - loss: 0.4156 - accuracy: 0.8207 - precision: 0.8353 - recall: 0.7889 - val_loss: 0.4469 - val_accuracy: 0.7708 - val_precision: 0.8421 - val_recall: 0.6667
Epoch 38/75
12/12 [==============================] - 1s 48ms/step - loss: 0.4139 - accuracy: 0.8315 - precision: 0.8590 - recall: 0.7701 - val_loss: 0.4456 - val_accuracy: 0.7708 - val_precision: 0.8421 - val_recall: 0.6667
Epoch 39/75
12/12 [==============================] - 1s 50ms/step - loss: 0.4015 - accuracy: 0.8315 - precision: 0.8511 - recall: 0.8247 - val_loss: 0.4341 - val_accuracy: 0.8021 - val_precision: 0.8222 - val_recall: 0.7708
Epoch 40/75
12/12 [==============================] - 1s 48ms/step - loss: 0.4168 - accuracy: 0.8315 - precision: 0.8

Epoch 73/75
12/12 [==============================] - 1s 48ms/step - loss: 0.2947 - accuracy: 0.9293 - precision: 0.9130 - recall: 0.9438 - val_loss: 0.3949 - val_accuracy: 0.8125 - val_precision: 0.8261 - val_recall: 0.7917
Epoch 74/75
12/12 [==============================] - 1s 49ms/step - loss: 0.3483 - accuracy: 0.8696 - precision: 0.8602 - recall: 0.8791 - val_loss: 0.3971 - val_accuracy: 0.8229 - val_precision: 0.8444 - val_recall: 0.7917
Epoch 75/75
12/12 [==============================] - 1s 52ms/step - loss: 0.3395 - accuracy: 0.8859 - precision: 0.9167 - recall: 0.8462 - val_loss: 0.3937 - val_accuracy: 0.8125 - val_precision: 0.8261 - val_recall: 0.7917


In [36]:
%tensorboard --logdir logs/scalars --host 'localhost'

Reusing TensorBoard on port 6006 (pid 8140), started 0:30:59 ago. (Use '!kill 8140' to kill it.)